In [129]:
import pandas as pd
import numpy as np
import pickle as pkl
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input,GRU, SimpleRNN
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [2]:
data = pd.read_csv('games.csv')

In [3]:
data.duplicated().sum()

429

In [4]:
data = data.drop_duplicates()

In [5]:
data.winner.value_counts()

white    9792
black    8919
draw      918
Name: winner, dtype: int64

In [6]:
data = data.drop(['white_id', 'black_id', 'white_rating', 'black_rating', 'id', 'rated'], axis=1)

In [7]:
data = pd.get_dummies(data, columns=['victory_status'], prefix='victory_status',  dtype=int)

In [8]:
data = data.drop(['created_at', 'last_move_at', 'increment_code', 'opening_name'], axis=1)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19629 entries, 0 to 20057
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   turns                     19629 non-null  int64 
 1   winner                    19629 non-null  object
 2   moves                     19629 non-null  object
 3   opening_eco               19629 non-null  object
 4   opening_ply               19629 non-null  int64 
 5   victory_status_draw       19629 non-null  int64 
 6   victory_status_mate       19629 non-null  int64 
 7   victory_status_outoftime  19629 non-null  int64 
 8   victory_status_resign     19629 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 1.5+ MB


In [10]:
top_n = 10  # Adjust as needed
top_categories = data['opening_eco'].value_counts().nlargest(top_n).index
data['opening_eco_processed'] = np.where(data['opening_eco'].isin(top_categories), data['opening_eco'], 'Other')

In [11]:
data = pd.get_dummies(data, columns=['opening_eco_processed'], prefix='opening_eco', dtype=int)

In [12]:
data = data.drop('opening_eco', axis=1)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19629 entries, 0 to 20057
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   turns                     19629 non-null  int64 
 1   winner                    19629 non-null  object
 2   moves                     19629 non-null  object
 3   opening_ply               19629 non-null  int64 
 4   victory_status_draw       19629 non-null  int64 
 5   victory_status_mate       19629 non-null  int64 
 6   victory_status_outoftime  19629 non-null  int64 
 7   victory_status_resign     19629 non-null  int64 
 8   opening_eco_A00           19629 non-null  int64 
 9   opening_eco_A40           19629 non-null  int64 
 10  opening_eco_B00           19629 non-null  int64 
 11  opening_eco_B01           19629 non-null  int64 
 12  opening_eco_B20           19629 non-null  int64 
 13  opening_eco_C00           19629 non-null  int64 
 14  opening_eco_C20       

In [14]:
moves = np.array(data.query("winner != 'draw'")['moves'])

In [15]:
labels = np.array(data.query("winner != 'draw'")['winner'].apply(lambda x: 1 if x =='white' else 0))
labels

array([1, 0, 1, ..., 1, 1, 0])

In [16]:
all_moves = set()
for moves_list in moves:
    for move in moves_list.split(' '):
        if move not in all_moves:
            all_moves.add(move)


vocab_length = len(all_moves)
vocab_length

4373

In [17]:
max_moves = data.turns.max()
max_moves

349

In [18]:
print(vocab_length)
print(max_moves)

4373
349


In [133]:
tokenizer = Tokenizer(num_words=vocab_length)
tokenizer.fit_on_texts(moves)

sequences = tokenizer.texts_to_sequences(moves)
word_index = tokenizer.word_index

model_inputs = pad_sequences(sequences, maxlen=max_moves)

pkl.dump(tokenizer, open('trained_models/tokenizer.pkl', 'wb'))

In [20]:
model_inputs.shape, labels.shape

((18711, 349), (18711,))

In [21]:
X_train,X_test,y_train,y_test = train_test_split(model_inputs, labels, test_size=0.2, random_state=42)

In [22]:
max_moves

349

In [28]:
model = Sequential()
model.add(Embedding(input_dim=vocab_length, output_dim=256, input_length=max_moves))
model.add(GRU(units=256))
model.add(Dense(1, activation='sigmoid'))

In [33]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=3, batch_size=32, validation_data=(X_test, y_test), verbose=2, callbacks=ReduceLROnPlateau())

Epoch 1/3
468/468 - 26s - loss: 0.2208 - accuracy: 0.9107 - val_loss: 0.2762 - val_accuracy: 0.8945 - lr: 0.0010 - 26s/epoch - 55ms/step
Epoch 2/3
468/468 - 12s - loss: 0.1813 - accuracy: 0.9272 - val_loss: 0.2721 - val_accuracy: 0.8990 - lr: 0.0010 - 12s/epoch - 25ms/step
Epoch 3/3
468/468 - 11s - loss: 0.1511 - accuracy: 0.9391 - val_loss: 0.3149 - val_accuracy: 0.8832 - lr: 0.0010 - 11s/epoch - 23ms/step


In [30]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

117/117 [==============================] - 2s 14ms/step - loss: 0.3293 - accuracy: 0.8715


In [131]:
pkl.dump(model, open('trained_models/model.pkl', 'wb'))